In [27]:
import tensorflow as tf
import pyodbc
import numpy as np

# with open('DATASET.npy', 'rb') as f:
#     targets = np.load(f)
#     notes = np.load(f)
with open('DATASET-MOY.npy', 'rb') as f:
    targets = np.load(f)
    notes = np.load(f)



In [28]:
train_dataset = tf.data.Dataset.from_tensor_slices((notes[0:250], targets[0:250]))
valid_dataset = tf.data.Dataset.from_tensor_slices((notes[251:350], targets[251:350]))

In [29]:
class Modele(tf.keras.Model):

    def __init__(self):
        super(Modele, self).__init__()
        self.conv_layer_one = tf.keras.layers.Conv1D(11, kernel_size=1, strides=1, activation='relu', padding='same', name='conv_layer_one')
        self.conv_layer_two = tf.keras.layers.Conv1D(11, kernel_size=3, strides=3, activation='relu', padding='same', name='conv_layer_two')
        self.flatten_layer = tf.keras.layers.Flatten(name='flatten_layer')
        self.input_layer = tf.keras.layers.Dense(11, activation='relu', name='input_layer')
        self.output_layer = tf.keras.layers.Dense(2, activation='softmax', name='output_layer')

    def call(self, notes):
        conv_layer_one = self.conv_layer_one(notes)
        conv_layer_two = self.conv_layer_two(conv_layer_one)
        flatten_layer = self.flatten_layer(conv_layer_two)
        input_layer = self.input_layer(flatten_layer)
        output_layer = self.output_layer(input_layer)
        return output_layer


In [30]:
model = Modele()
# model.compile(
#     loss="sparse_categorical_crossentropy",
#     optimizer="sgd",
#     metrics=["accuracy"]
# )
# model.fit(notes[20:120], targets[20:120], epochs=5)
# model.save("model_test", save_format="tf")



In [31]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [32]:
# loss "L'erreur"
train_loss = tf.keras.metrics.Mean(name="train_loss")
valid_loss = tf.keras.metrics.Mean(name="valid_loss")
# Accuracy "La précision"
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")
valid_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="test_accuracy")

In [33]:
@tf.function
def train_step(note, target):
    with tf.GradientTape() as tape:
        # faire une prediction sur tous le batch (groupe ou patie)
        prediction = model(note, training=True)
        # calculer l'erreur sur cette prediction
        loss = loss_object(target, prediction)
    # calcule le gradient qui respecte l'erreur  
    gradients = tape.gradient(loss, model.trainable_variables)
    # optimiser les poids du model selon le gradient calculé
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(target, prediction)

In [34]:
@tf.function
def test_step(note, target):
    # faire une prediction sur tous le batch (groupe ou patie)
    prediction = model(note)
    # calculer l'erreur sur cette prediction
    loss = loss_object(target, prediction)
    valid_loss(loss)
    valid_accuracy(target, prediction)

In [35]:
epoch= 10
batch_size = 10
b =0
model.compile(loss=loss_object, optimizer=optimizer)
for epoch in range(epoch):
    
    for notes_batch, tagets_batch in train_dataset.batch(batch_size):
        train_step(notes_batch, tagets_batch)
        template = '\r batch: {}/{}, erreur:  {}, Précision: {}'
        print(template.format(b, len(targets), train_loss.result(), train_accuracy.result()*100))
        b+= batch_size
        
        
    for notes_batch, tagets_batch in valid_dataset.batch(batch_size):
        test_step(notes_batch, tagets_batch)
        template = '\r epoch: {}, Test erreur:  {}, Test Précision: {}'
        print(template.format(epoch+1, valid_loss.result(), valid_accuracy.result()*100))
        
    print("------------")
    train_loss.reset_states()
    train_accuracy.reset_states()
    valid_loss.reset_states()
    valid_accuracy.reset_states()

model.save("modele", save_format="tf")
# tf.saved_model.save(model, "saved_model_path") 

 batch: 0/388, erreur:  5.21882438659668, Précision: 40.0
 batch: 10/388, erreur:  4.011632919311523, Précision: 50.0
 batch: 20/388, erreur:  3.091228485107422, Précision: 60.000003814697266
 batch: 30/388, erreur:  2.76322078704834, Précision: 62.5
 batch: 40/388, erreur:  2.5612101554870605, Précision: 64.0
 batch: 50/388, erreur:  2.223525285720825, Précision: 68.33333587646484
 batch: 60/388, erreur:  2.163465738296509, Précision: 67.14285278320312
 batch: 70/388, erreur:  1.9955203533172607, Précision: 68.75
 batch: 80/388, erreur:  2.0088348388671875, Précision: 67.77777862548828
 batch: 90/388, erreur:  2.0040650367736816, Précision: 66.0
 batch: 100/388, erreur:  1.9616512060165405, Précision: 65.45454406738281
 batch: 110/388, erreur:  1.9223304986953735, Précision: 65.0
 batch: 120/388, erreur:  1.9814484119415283, Précision: 63.846153259277344
 batch: 130/388, erreur:  2.0209338665008545, Précision: 61.42856979370117
 batch: 140/388, erreur:  1.9952479600906372, Précision: 

 batch: 910/388, erreur:  0.6635031700134277, Précision: 58.235294342041016
 batch: 920/388, erreur:  0.6654157042503357, Précision: 57.22222137451172
 batch: 930/388, erreur:  0.6620609760284424, Précision: 58.421051025390625
 batch: 940/388, erreur:  0.6633265614509583, Précision: 58.499996185302734
 batch: 950/388, erreur:  0.6679269671440125, Précision: 57.619049072265625
 batch: 960/388, erreur:  0.6652589440345764, Précision: 58.63636016845703
 batch: 970/388, erreur:  0.6710842847824097, Précision: 58.26087188720703
 batch: 980/388, erreur:  0.6693646311759949, Précision: 57.916664123535156
 batch: 990/388, erreur:  0.6704597473144531, Précision: 56.400001525878906
 epoch: 4, Test erreur:  0.7703854441642761, Test Précision: 40.0
 epoch: 4, Test erreur:  0.7126941680908203, Test Précision: 45.0
 epoch: 4, Test erreur:  0.706852912902832, Test Précision: 46.66666793823242
 epoch: 4, Test erreur:  0.698976993560791, Test Précision: 50.0
 epoch: 4, Test erreur:  0.6878408193588257,

 epoch: 7, Test erreur:  0.7078845500946045, Test Précision: 55.0
 epoch: 7, Test erreur:  0.7020270824432373, Test Précision: 53.333335876464844
 epoch: 7, Test erreur:  0.694237470626831, Test Précision: 57.5
 epoch: 7, Test erreur:  0.6841708421707153, Test Précision: 60.000003814697266
 epoch: 7, Test erreur:  0.671425998210907, Test Précision: 63.33333206176758
 epoch: 7, Test erreur:  0.6662107110023499, Test Précision: 64.28571319580078
 epoch: 7, Test erreur:  0.6728719472885132, Test Précision: 61.25
 epoch: 7, Test erreur:  0.6722007989883423, Test Précision: 62.222225189208984
 epoch: 7, Test erreur:  0.6822808384895325, Test Précision: 60.60606384277344
------------
 batch: 1750/388, erreur:  0.800457775592804, Précision: 40.0
 batch: 1760/388, erreur:  0.7325446605682373, Précision: 50.0
 batch: 1770/388, erreur:  0.6880574226379395, Précision: 60.000003814697266
 batch: 1780/388, erreur:  0.6576465964317322, Précision: 62.5
 batch: 1790/388, erreur:  0.6423500776290894, P

INFO:tensorflow:Assets written to: modele\assets


In [36]:
load_model = tf.keras.models.load_model("modele")
pred = load_model.predict(notes[366:367])
print(pred, targets[366:367])
print(notes[366:367])

[[0.5008455  0.49915454]] [0]
[[[ 2.    9.5   8.25  9.5  11.25  5.75  6.5  11.   11.5  10.    4.  ]
  [10.   10.   10.25  9.    8.   12.25 14.   14.75 15.    0.   11.5 ]
  [ 0.    0.    0.    0.   13.5   0.    0.   13.    0.    0.    0.  ]]]
